In [1]:
import numpy as np
import pandas as pd
import random
import itertools
import matplotlib.pyplot as plt
import plotly.express as px
import glob
import spacy
import sklearn
from collections import defaultdict, Counter
from bisect import bisect_left
import matplotlib.pyplot as plt
import matplotlib
import io
import base64
import pymp
from multiprocessing import Pool, Manager, cpu_count
from IPython.display import display_html
from itertools import chain, cycle
import lightgbm as lgb
import sys
sys.path.append('../../../../utils')
sys.path.append('..')
import gezi
from gezi import tqdm
from src.eval import *
from src.util import *
from src import config
from src.visualize import *
from src.rewards import *
pd.set_option('display.float_format', lambda x: '%.02f' % x)

No module named 'tensorflow.python.keras.layers.preprocessing'


In [2]:
# https://www.kaggle.com/chasembowers/sequence-postprocessing-v2-67-lb/notebook#Sequence-Datasets

In [3]:
root = '../input/feedback-prize-2021'

In [4]:
df = pd.read_feather(f'{root}/train_en.fea')
df_gt = pd.read_csv('../working/offline/60/valid_gt.csv')

In [5]:
MAX_SEQ_LEN = {}
df['len'] = df.end - df.start
max_lens = df.groupby('para_type')['len'].quantile(.995)
for i in range(len(ALL_CLASSES)):
  MAX_SEQ_LEN[id2dis[i]] = int(max_lens[i])
MAX_SEQ_LEN

{'Nothing': 132,
 'Claim': 55,
 'Evidence': 299,
 'Position': 59,
 'Concluding Statement': 191,
 'Lead': 197,
 'Counterclaim': 95,
 'Rebuttal': 116}

In [6]:
d = pd.read_feather(f'{root}/para_label.fea')

In [7]:
gts = {}
for row in tqdm(d.itertuples(), total=len(d)):
  gts[row.id] = decode_label_all(row.start, row.para_types)

  0%|          | 0/15594 [00:00<?, ?it/s]

In [8]:
folds = {}
for row in tqdm(df.itertuples(), total=len(df)):
  folds[row.id] = row.kfold

  0%|          | 0/175160 [00:00<?, ?it/s]

In [9]:
np.random.seed(12345)

In [10]:
subfolds = {}
for row in tqdm(df.itertuples(), total=len(df)):
  subfolds[row.id] = np.random.randint(5)

  0%|          | 0/175160 [00:00<?, ?it/s]

In [11]:
from src.ensemble_conf import v, mns
ic(v, mns)

ic| ensemble_conf.py:136 in <module>
    mns: ['deberta-v3.start.nwemb-0.mark_end-0',
          'deberta-xlarge.start',
          'deberta-v3.start.len1024.stride-256.seq_encoder-0',
          'deberta-v3.start.len1024.rnn_layers-2',
          'deberta-v3.start.len1280.rnn_type-GRU',
          'deberta-v3.end.len1024',
          'deberta-v3.start.len1024.rnn_type-GRU',
          'deberta-v3.end.len1024.rnn_type-GRU',
          'deberta-v3.start.len1280.rnn_layers-2',
          'funnel.end.len1024',
          'deberta-v3.end.len1024.seq_encoder-0',
          'deberta-v3.start.weight_decay',
          'deberta.end',
          'deberta-v3.end.len1536.seq_encoder-0',
          'funnel.start.len1024',
          'bart.end.run2',
          'deberta-v3.end',
          'roberta.start',
          'deberta-v3.start.len1600.rnn_layers-2',
          'longformer.start.len1600',
          'deberta-v3.se',
          'deberta-v3.start.nwemb-0',
          'bart.se2',
          'longformer.start.len1536'

(60,
 ['deberta-v3.start.nwemb-0.mark_end-0',
  'deberta-xlarge.start',
  'deberta-v3.start.len1024.stride-256.seq_encoder-0',
  'deberta-v3.end.len1024.seq_encoder-0',
  'deberta-v3.se',
  'longformer.start.len1536',
  'deberta-xlarge.end',
  'deberta-v3.start.len1536',
  'deberta-v3.start.len1024.stride-256',
  'deberta-v3.start.len1024.rnn_bi',
  'deberta-v3.se2',
  'deberta-v3.start.stride-256.seq_encoder-0',
  'deberta.start',
  'roberta.start.nwemb-0',
  'bart.start.run2',
  'deberta-v3.mid.len1024'])

In [12]:
info = gezi.load(f'../working/offline/{v}/valid.pkl')
info = gezi.batch2list(info)
info = gezi.sort_list_byid(info)

In [13]:
votes = gezi.read_pickle(f'../working/offline/{v}/votes.pkl')

In [14]:
ids = set(df.id)

In [15]:
start_dict, se_dict, sec_dict = {}, {}, {}
for id in tqdm(ids):
  start_dict[id] = {}
  se_dict[id] = defaultdict(set)
  sec_dict[id] = defaultdict(set)
  for i, vote in enumerate(votes):
    for item in vote[id]:
      start, end = item['start'], item['end']
      cls_ = item['cls']
      start_dict[id][start] = 1
      se_dict[id][(start, end)].add(cls_)
      sec_dict[id][(start, end, cls_)].add(i)

  0%|          | 0/15594 [00:00<?, ?it/s]

In [158]:
def para_dataset(fold=None, subfold=None, infer=False):
  total = len(info)
  fes = []
  scores = []
  matches = []
  for i in tqdm(range(total)):
    # if i > 1:
    #   break
    x = info[i]
    id = x['id']
    gt = gts[id]
    fe = {}
    fe['id'] = id
    fe['index'] = i
    fe['fold'] = folds[id]
    if fold is not None and fe['fold'] != fold:
      continue
    fe['subfold'] = subfolds[id]
    if subfold is not None and fe['subfold'] != subfold:
      continue
    fe['num_words'] = x['num_words']
    fe['seps'] = (x['start_probs'][:,1] > 0.5).sum()
    fe['sep_ratio'] = fe['seps'] / fe['num_words']
    num_words = x['num_words']
    # ic(num_words, start_dict[id], se_dict[id])
    for j in range(num_words):
      if j not in start_dict[id]:
        continue
      fe['start'] = j
      fe['start_ratio'] = (j + 1) / num_words
      fe['start_probs'] = x['probs'][j]
      fe['start_sep_prob'] = x['start_probs'][j][1] if j > 0 else 1.
      preds = {k: 0 for k in range(NUM_CLASSES)}
      fe['start_cls'] = x['preds'][j]
      probs = x['probs'][j]
      fe['start_cls_prob'] = probs[fe['start_cls']]
      fe['max_start_prob'] = probs.max()
      preds[x['preds'][j]] += 1
      fe['pre_cls'] = -1 if j == 0 else x['preds'][j - 1]
      fe['start_diff'] = int(fe['pre_cls'] != fe['start_cls'])
      fe['pre_max_prob'] = 1 if j == 0 else x['probs'][j - 1].max()
      fe['pre_cls_prob'] = 1 if j == 0 else x['probs'][j - 1][fe['pre_cls']]
      sep_count = 0
      for k in range(j + 1, num_words):
        if (j, k + 1) not in se_dict[id]:
          continue
        probs = x['probs'][j: k + 1].sum(0)
        fe['para_len'] = k + 1 - j
        fe['para_len_ratio'] = (k + 1 - j) / num_words
        preds[x['preds'][k]] += 1  
        fe['end'] = k + 1
        fe['end_ratio'] = (k + 1) / num_words
        fe['end_probs'] = x['probs'][k]
        fe['max_end_prob'] = x['probs'][k + 1].max() if k + 1 < num_words else 1
        end_cls = np.argmax(x['probs'][k])
        fe['end_cls'] = end_cls
        fe['end_cls_prob'] = x['probs'][k][end_cls]
        fe['end_sep_prob'] = x['start_probs'][k + 1][1] if k + 1 < num_words else 1.
        fe['sep_add_prob'] = (fe['start_sep_prob'] + fe['end_sep_prob']) / 2.
        fe['sep_mul_prob'] = (fe['start_sep_prob'] * fe['end_sep_prob']) ** 0.5
        fe['num_classes'] = len([k for k in range(NUM_CLASSES) if preds[k] > 0])
        mean_probs = gezi.softmax(probs)
        fe['mean_probs'] = mean_probs
        fe['max_prob'] = mean_probs.max()
        top_classes = np.argsort(-mean_probs,axis=0)[:2]
        fe['top_class'] = top_classes[0]
        fe['top_class2'] = top_classes[1]
        fe['next_cls'] = -1 if k + 1  == num_words else x['preds'][k + 1]
        fe['end_diff'] = int(fe['next_cls'] != fe['end_cls'])
        fe['next_max_prob'] = 1 if k + 1  == num_words else x['probs'][k + 1].max()
        # top_classes = top_classes[:1]

        for cls in range(len(ALL_CLASSES)):
          # if cls not in se_dict[id][(j, k + 1)]:
          #   continue
          if fe['mean_probs'].argmax() != cls:
            continue
          for idx in range(len(votes)):
            fe[f'model{idx}'] = 0
          model_matches = 0
          for idx in sec_dict[id][(j, k + 1, cls)]:
            fe[f'model{idx}'] = 1
            model_matches += 1
          fe['model_matches'] = model_matches / len(votes)
          fe['mean_prob'] = fe['mean_probs'][cls]
          if fe['mean_prob'] < proba_thresh[id2dis[cls]]:
            continue
          fe['start_prob'] = x['probs'][j][cls]
          fe['end_prob'] = x['probs'][k][cls]
          fe['class_ratio'] = (x['preds'][j:k+1] == cls).sum() / fe['para_len']
          # fe['class_max_prob'] = x['probs'][j:k+1][cls].max()
          # fe['class_min_prob'] = x['probs'][j:k+1][cls].min()
          fe['start_eq'] = int(fe['start_cls'] == fe['pre_cls'])
          fe['start_eq2'] = int(cls == fe['pre_cls'])
          fe['end_eq'] = int(fe['end_cls'] == fe['next_cls'])
          fe['end_eq2'] = int(cls == fe['next_cls'])
          fe['cls'] = cls
          fe['is_top_class'] = int(cls == fe['top_class'])
          fe['is_top_class2'] = int(cls == fe['top_class2'])
          if not infer:
            fe['score'] = best_match(gt[cls], [j, k + 1])
            fe['match'] = calc_match(gt[cls], [[j, k + 1]])

            scores.append(fe['score'])
            matches.append(fe['match'])
          fes.append(fe.copy())

  # if not infer:
  #   ic(len(scores), len(matches))
  #   ic(np.mean(scores), np.mean(matches))
  d = pd.DataFrame(fes)
  return d

In [159]:
dall = para_dataset()

  0%|          | 0/15594 [00:00<?, ?it/s]

In [160]:
dall

,id,index,fold,subfold,num_words,seps,sep_ratio,start,start_ratio,start_probs,...,class_ratio,start_eq,start_eq2,end_eq,end_eq2,cls,is_top_class,is_top_class2,score,match
0,0000D23A521A,0,0,0,251,9,0.04,0,0.00,"[0.001917590759271003, 0.0019244975630331909, ...",...,1.00,0,0,1,1,5,1,0,0.00,0
1,0000D23A521A,0,0,0,251,9,0.04,0,0.00,"[0.001917590759271003, 0.0019244975630331909, ...",...,1.00,0,0,0,0,5,1,0,0.00,0
2,0000D23A521A,0,0,0,251,9,0.04,16,0.07,"[0.007096823139469225, 0.0038828863716732975, ...",...,0.72,1,0,0,0,3,1,0,0.53,1
3,0000D23A521A,0,0,0,251,9,0.04,21,0.09,"[0.0030103651257567298, 0.04211587626663227, 0...",...,1.00,0,0,0,0,3,1,0,0.38,0
4,0000D23A521A,0,0,0,251,9,0.04,21,0.09,"[0.0030103651257567298, 0.04211587626663227, 0...",...,0.68,0,0,1,0,3,1,0,0.33,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
492043,FFFF80B8CC2F,15593,3,3,168,2,0.01,60,0.36,"[0.00846240406675545, 0.1874541361694501, 0.75...",...,1.00,1,1,1,1,2,1,0,0.12,0
492044,FFFF80B8CC2F,15593,3,3,168,2,0.01,63,0.38,"[0.004458047269476028, 0.20680522259614403, 0....",...,1.00,1,1,1,1,2,1,0,0.10,0
492045,FFFF80B8CC2F,15593,3,3,168,2,0.01,80,0.48,"[0.013228141344881891, 0.07948528910714266, 0....",...,1.00,1,1,1,1,2,1,0,0.11,0
492046,FFFF80B8CC2F,15593,3,3,168,2,0.01,80,0.48,"[0.013228141344881891, 0.07948528910714266, 0....",...,1.00,1,1,0,0,2,1,0,0.46,0


In [161]:
dvalid = dall[dall.fold==0]
dtrain = dall[dall.fold!=0]

In [226]:
# dtrain = d[d.subfold != 0]
# # dtrain = d0[d0.subfold == 1]
# dvalid = d[d.subfold == 0]
reg_cols =  [
              'num_words', 
              'start', 
              'start_ratio', 
              'end', 
              'end_ratio',
              'start_sep_prob', 
              'end_sep_prob', 
              'para_len', 
              'para_len_ratio',
              'num_classes', 
              'mean_prob', 
              'start_prob', 
              'end_prob', 
              'start_eq',
              'start_eq2',
              'end_eq',
              'end_eq2',
              # 'seps', 'sep_ratio', 
              'class_ratio',
              'max_prob',
              'model_matches',
              # 'sep_add_prob', 
              # 'sep_mul_prob', 
              # 'is_top_class', 
              # 'is_top_class2',
              # 'class_max_prob', 'class_min_prob'
              # 'start_cls_prob',
              # 'pre_cls_prob',
              # 'end_cls_prob',
              # 'start_diff',
              # 'end_diff',
        ]
for idx in range(len(votes)):
  reg_cols += [f'model{idx}']
cat_cols = [
            'cls', 
            'start_cls', 
            'end_cls',
            # 'top_class',
            'pre_cls', 
            'next_cls'
            ]
# label_col = 'match'
label_col = 'score'
cols = reg_cols + cat_cols
X_train = dtrain[cols]
y_train = dtrain[[label_col]]

In [227]:
X_valid = dvalid[cols]
# y_valid = dvalid[[label_col]]
y_valid = dvalid[['match']]

In [228]:
learning_rate = 0.1
num_boost_round = 2000
params = {
          # "objective": "binary",
          "objective": "regression" if label_col is 'score' else 'binary',
          # "objective": "cross_entropy",
          "metric": "auc",
          "boosting_type": "gbdt",
          "learning_rate": learning_rate,
          "num_leaves": 6,
          "max_bin": 256,
          "feature_fraction": 0.9,
          "verbosity": 0,
          "drop_rate": 0.1,
          "is_unbalance": True,
          "max_drop": 50,
          "min_child_samples": 200,
          "min_child_weight": 150,
          "min_split_gain": 0,
          "bagging_freq": 5,
          "bagging_fraction": 0.9,
          # "num_trees": 200,
          "subsample": 0.9
          }

In [229]:
d_train = lgb.Dataset(X_train, y_train)
d_valid = lgb.Dataset(X_valid, y_valid, reference=d_train)
bst = lgb.train(params, d_train, num_boost_round, valid_sets=d_valid, 
                categorical_feature=cat_cols,
                verbose_eval=1,
                early_stopping_rounds=10)

[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=0.9 will be ignored. Current value: bagging_fraction=0.9
[LightGBM] [Warning] Met negative value in categorical features, will convert it to NaN
[LightGBM] [Warning] Met negative value in categorical features, will convert it to NaN
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=0.9 will be ignored. Current value: bagging_fraction=0.9
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.719143 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=0.9 will be ignored. Current value: bagging_fraction=0.9
[1]	valid_0's auc: 0.715275
Training until validation scores don't improve for 10 rounds
[2]	valid_0's auc: 0.724499
[3]	valid_0's auc: 0.735714
[4]	valid_0's auc: 0.741155
[5]	valid_0's auc: 0.742532
[6]	valid_0's auc: 0.748307
[7]	valid_0's a

In [230]:
l = list(zip(bst.feature_name(), bst.feature_importance()))
l.sort(key=lambda x: -x[1])
l

[('end_sep_prob', 241),
 ('cls', 238),
 ('end_prob', 236),
 ('next_cls', 220),
 ('start_prob', 181),
 ('para_len_ratio', 162),
 ('num_words', 154),
 ('start_sep_prob', 131),
 ('para_len', 120),
 ('end_ratio', 119),
 ('pre_cls', 116),
 ('model_matches', 104),
 ('start_ratio', 99),
 ('mean_prob', 98),
 ('start_cls', 95),
 ('end_cls', 89),
 ('start', 77),
 ('class_ratio', 75),
 ('end', 72),
 ('model14', 23),
 ('start_eq2', 21),
 ('end_eq2', 20),
 ('model1', 16),
 ('max_prob', 13),
 ('model3', 12),
 ('model12', 11),
 ('model8', 8),
 ('model11', 8),
 ('end_eq', 5),
 ('model2', 5),
 ('model0', 4),
 ('model5', 3),
 ('model10', 3),
 ('model7', 2),
 ('start_eq', 1),
 ('model4', 1),
 ('model6', 1),
 ('model9', 1),
 ('num_classes', 0),
 ('model13', 0),
 ('model15', 0),
 ('model16', 0)]

In [231]:
def greedy_decodes(df, cols, para_classifier):
  df['pred'] = para_classifier.predict(df[cols])
  ids = set(df.id)
  ids_list, types_list, preds_list = [], [], []
  for id in tqdm(ids):
    d = df[df.id == id]
  #   max_idx, max_score = 0, -1
  #   for i in range(len(votes)):
  #     d_ = d[d[f'model{i}'] > 0]
  #     if len(d_):
  #       score = d_.pred.mean()
  #       if score > max_score:
  #         max_score = score
  #         max_idx = i
  #   d = d[d[f'model{max_idx}'] > 0]
    if len(d):
      d = d.sort_values(['pred'], ascending=[False])
      num_words = d.num_words.values[0]
      used = np.zeros(num_words)
      for row in d.itertuples():
        # if row.model0 < 1 and row.pred < 0.9:
        # #   continue
        # if row.model0 < 1 and not row.cls in [2]:
        #   continue
        start, end = row.start, row.end
        # cls = row.cls
        cls = row.cls
        # if not cls:
        #   continue
        # if used[start: end].sum() == 0:
        # if row.model0 < 1 or row.pred < 0.9:
        #   continue
        if used[start: end].sum() / (end - start) < 0.01:
          if row.mean_prob > proba_thresh[id2dis[cls]]:
            used[start: end] = 1
            if cls:
              ids_list.append(id)
              types_list.append(id2dis[cls])
              preds_list.append(' '.join([str(x) for x in range(start, end)]))
        if used.sum() == num_words:
          break
  df = pd.DataFrame({
    'id': ids_list,
    'class': types_list,
    'predictionstring': preds_list,
  })
  return df

In [232]:
df_pred = greedy_decodes(dvalid, cols, bst)

  0%|          | 0/3094 [00:00<?, ?it/s]

In [233]:
dv = dvalid[['id', 'start', 'end', 'cls', 'start_prob', 'end_prob', 'model0', 'score', 'match', 'pred']]

In [234]:
id = list(set(dv.id))[3]

In [235]:
dv[dv.id==id]

,id,start,end,cls,start_prob,end_prob,model0,score,match,pred
350852,B659CCF9A5C5,0,8,3,0.99,0.99,1,0.57,1,0.28
350853,B659CCF9A5C5,8,17,1,0.41,0.97,0,0.33,0,0.34
350854,B659CCF9A5C5,9,17,1,0.95,0.97,1,0.38,0,0.21
350855,B659CCF9A5C5,18,26,1,0.97,0.98,1,0.89,1,0.85
350856,B659CCF9A5C5,26,48,1,0.94,0.93,1,1.00,1,0.85
350857,B659CCF9A5C5,48,133,2,0.90,0.96,1,1.00,1,0.79
350858,B659CCF9A5C5,133,147,1,0.64,0.63,0,0.32,0,0.42
350859,B659CCF9A5C5,133,177,2,0.34,0.94,0,0.00,0,0.50
350860,B659CCF9A5C5,133,178,2,0.34,0.66,0,0.01,0,0.45
350861,B659CCF9A5C5,133,183,2,0.34,0.65,0,0.08,0,0.45


In [236]:
df_pred

,id,class,predictionstring
0,38DD98A04439,Evidence,156 157 158 159 160 161 162 163 164 165 166 16...
1,38DD98A04439,Concluding Statement,184 185 186 187 188 189 190 191 192 193 194 19...
2,38DD98A04439,Claim,49 50 51
3,38DD98A04439,Evidence,62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 7...
4,38DD98A04439,Evidence,113 114 115 116 117 118 119 120 121 122 123 12...
...,...,...,...
27893,1B065440759E,Evidence,197 198 199 200 201 202 203 204 205 206 207 20...
27894,1B065440759E,Claim,311 312 313 314 315 316 317 318
27895,1B065440759E,Evidence,319 320 321 322 323 324 325 326 327 328 329 33...
27896,1B065440759E,Evidence,120 121 122 123 124 125 126 127 128 129 130 13...


In [237]:
res = calc_metrics(df_gt[df_gt.id.isin(set(dvalid.id))], df_pred)
ic(res['f1/Overall'])
ic(res)

calc_f1:   0%|          | 0/7 [00:00<?, ?it/s]

ic| 3430601751.py:2 in <module>- res['f1/Overall']: 0.7103608015083883
ic| 3430601751.py:3 in <module>
    res: {'acc/Claim': 0.7114007906827653,
          'acc/Concluding Statement': 0.8434720739950196,
          'acc/Counterclaim': 0.6193672099712368,
          'acc/Evidence': 0.7592859518793658,
          'acc/Lead': 0.8286020353508302,
          'acc/Position': 0.7348157982077663,
          'acc/Rebuttal': 0.5327455919395466,
          'f1/Claim': 0.6845920518225284,
          'f1/Concluding Statement': 0.865012769062386,
          'f1/Counterclaim': 0.5902238465052535,
          'f1/Evidence': 0.7561002539472231,
          'f1/Lead': 0.8375744450460205,
          'f1/Overall': 0.7103608015083883,
          'f1/Position': 0.7287689269256089,
          'f1/Rebuttal': 0.5102533172496985,
          'recall/Claim': 0.6597304795877923,
          'recall/Concluding Statement': 0.8876825159116436,
          'recall/Counterclaim': 0.5636998254799301,
          'recall/Evidence': 0.75294117

{'f1/Overall': 0.7103608015083883,
 'f1/Claim': 0.6845920518225284,
 'acc/Claim': 0.7114007906827653,
 'recall/Claim': 0.6597304795877923,
 'f1/Evidence': 0.7561002539472231,
 'acc/Evidence': 0.7592859518793658,
 'recall/Evidence': 0.7529411764705882,
 'f1/Position': 0.7287689269256089,
 'acc/Position': 0.7348157982077663,
 'recall/Position': 0.722820763956905,
 'f1/Concluding Statement': 0.865012769062386,
 'acc/Concluding Statement': 0.8434720739950196,
 'recall/Concluding Statement': 0.8876825159116436,
 'f1/Lead': 0.8375744450460205,
 'acc/Lead': 0.8286020353508302,
 'recall/Lead': 0.8467432950191571,
 'f1/Counterclaim': 0.5902238465052535,
 'acc/Counterclaim': 0.6193672099712368,
 'recall/Counterclaim': 0.5636998254799301,
 'f1/Rebuttal': 0.5102533172496985,
 'acc/Rebuttal': 0.5327455919395466,
 'recall/Rebuttal': 0.4895833333333333}